In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy.sparse.linalg import svds
from tqdm import tqdm
from collections import Counter
from itertools import chain, combinations
from tqdm import tqdm 
import time

In [13]:
tqdm.pandas()

In [2]:
#플레이 리스트
train = pd.read_json('data/train.json')
#곡 정보
song_meta = pd.read_json('data/song_meta.json')

In [3]:
# 해당 노래를 부른 가수의 수
song_meta['singer_cnt'] = song_meta['artist_name_basket'].apply(lambda x: len(x))

In [4]:
# 모든 장르 리스트
all_genre = list(set(np.concatenate(song_meta['song_gn_gnr_basket'])))

In [5]:
# 단체곡 가수를 분리하기 위해 flatten
artist_flat = pd.DataFrame([[i, x] for i, y in song_meta['artist_name_basket'].iteritems() for x in y], columns=['i','artist'])
artist_flat = artist_flat.set_index('i')

In [6]:
# 단체곡을 한 가수씩 펼친 dataframe
song_meta_artist = song_meta.merge(artist_flat, left_index=True, right_index=True).reset_index(drop = True)

In [9]:
# count vectorizer로 세기위해 장르를 공백으로 구분된 string으로 만든다.
song_meta_artist['words'] = song_meta_artist['song_gn_gnr_basket'].apply(lambda x : " ".join(x))

In [10]:
def add_str(x):
    t = []
    t.append(x)
    return " ".join(x)

In [14]:
# 가수별로 그룹바이한 장르리스트를 합쳐 공백으로 구분된 string으로 만든다.
art_gnr = song_meta_artist.groupby('artist')['words'].progress_apply(add_str)

100%|███████████████████████████████████████████████████████████████████████| 111304/111304 [00:03<00:00, 29341.66it/s]


In [16]:
art_gnr

artist
                                       GN1200 GN2000 GN1100
 A.R.D.I.                                            GN1100
 Al Kasha                                     GN1500 GN2200
 Benny Benassi    GN1100 GN1100 GN1100 GN1100 GN1100 GN1100
 Black Pata                                   GN1100 GN1100
                                    ...                    
힙합시보사동                                        GN0500 GN0300
힛 더 나인                                        GN0400 GN0200
＝AH15                                                GN1400
？班生                                                  GN0900
￦HERE AM I                                    GN0500 GN0300
Name: words, Length: 111304, dtype: object

In [17]:
art_gnr = pd.DataFrame(art_gnr)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
model = CountVectorizer()
artist_gnr_vector = model.fit_transform(art_gnr['words'])

In [21]:
artist_gnr_vector.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [24]:
model.get_feature_names()

['gn0100',
 'gn0200',
 'gn0300',
 'gn0400',
 'gn0500',
 'gn0600',
 'gn0700',
 'gn0800',
 'gn0900',
 'gn1000',
 'gn1100',
 'gn1200',
 'gn1300',
 'gn1400',
 'gn1500',
 'gn1600',
 'gn1700',
 'gn1800',
 'gn1900',
 'gn2000',
 'gn2100',
 'gn2200',
 'gn2300',
 'gn2400',
 'gn2500',
 'gn2600',
 'gn2700',
 'gn2800',
 'gn2900',
 'gn9000']

In [50]:
len(model.get_feature_names())

30

In [33]:
artist_list = art_gnr.index.tolist()

In [36]:
artist_gnr_df = pd.DataFrame(artist_gnr_vector.toarray(), index =artist_list, columns = model.get_feature_names())

In [65]:
artist_gnr_df.loc[['아이유']]

,gn0100,gn0200,gn0300,gn0400,gn0500,gn0600,gn0700,gn0800,gn0900,gn1000,...,gn2100,gn2200,gn2300,gn2400,gn2500,gn2600,gn2700,gn2800,gn2900,gn9000
아이유,91,37,0,3,0,23,0,4,0,0,...,0,0,0,0,0,1,0,0,0,0


In [61]:
artist_gnr_df.to_csv('data/artist_gnr_df.csv')